In [1]:
import re
import numpy as np

In [2]:
class Target:
    regex = re.compile('.*target area: x=(-?[0-9]+)..(-?[0-9]+), y=(-?[0-9]+)..(-?[0-9]+).*')
    
    def __init__(self, f):
        match = self.regex.match(f.readline().rstrip())
        data = tuple(int(match.group(i)) for i in range(1, 5))
        self.tx = (data[0], data[1])
        self.ty = (data[2], data[3])
        self.values = set([(i, j) for i in range(self.tx[0], self.tx[1]+1) for j in range (self.ty[0], self.ty[1]+1)])

In [3]:
def max_y(vx, vy, target):
    x, y = 0, 0
    ymax = y
    while True:
        x += vx
        y += vy
        
        vx -= np.sign(vx)
        vy -= 1
        if y > ymax:
            ymax = y
        if (x, y) in target.values:
            return ymax
        elif (x > target.tx[1] and vx >= 0) or (x < target.tx[0] and vx <= 0) or (y < target.ty[0] and vy <= 0):
            return None

In [4]:
def launch_high(target):
    vx = 0
    if target.tx[0] <= 0 <= target.tx[1]:
        vx = 0
    elif target.tx[1] < 0:
        while vx * (vx - 1) // 2 > target.tx[1]:
            vx -= 1
    elif target.tx[0] > 0:
        while vx * (vx + 1) // 2 < target.tx[0]:
            vx += 1
    vy = 10 * np.abs(target.ty[1] - target.ty[0])
    ymax = max_y(vx, vy, target)
    while ymax is None:
        vy -= 1
        ymax = max_y(vx, vy, target)
    return ymax

In [5]:
def find_all_launches(target):
    if target.tx[0] <= 0 <= target.tx[1]:
        vxrange = range(target.tx[0], target.tx[1]+1)
    elif target.tx[1] < 0:
        vxrange = range(target.tx[0], 0)
    elif target.tx[0] > 0:
        vxrange = range(target.tx[1] + 1)
    
    ydiff = np.abs(target.ty[1] - target.ty[0])
    if target.ty[0] < 0:
        vyrange = range(target.ty[0], target.ty[1] + 10 * ydiff)
    else:
        vyrange = range(target.ty[1] + 20 * ydiff)
    
    launches = []
    for vx in vxrange:
        for vy in vyrange:
            if max_y(vx, vy, target) is not None:
                launches.append((vx, vy))
    return launches

In [6]:
with open('input.txt', 'r') as f:
    target = Target(f)

In [7]:
print(f'Part 1: {launch_high(target)}')

Part 1: 4560


In [8]:
print(f'Part 2: {len(find_all_launches(target))}')

Part 2: 3344
